# Node A

Import

In [1]:
import rospy
from actionlib import SimpleActionClient
from assignment_2_2022.msg import PlanningAction, PlanningGoal, Message1
from assignment_2_2022.srv import Service1
from jupyros.ros1 import ros3d
from ipywidgets import widgets, Layout, HBox
from IPython.display import display
from matplotlib import pyplot as plt
import threading
rospy.init_node('node_a', anonymous=True)

Define widgets

In [ ]:
x_new = widgets.FloatText(value=0.0, description='x:')
y_new = widgets.FloatText(value=0.0, description='y:')
set_goal = widgets.Button(description='Set goal')
cancel_goal = widgets.Button(description='Cancel goal')
robot_position = widgets.Text(description='Robot position:')
target_position = widgets.Text(description='Target position:')
distance_obstacle = widgets.Text(description='Distance to obstacle:')

Buttons for setting and cancelling the goal

In [ ]:
def set_goal_clicked(b):
    target = PlanningGoal()
    target.target_pose.pose.position.x = x_new.value
    target.target_pose.pose.position.y = y_new.value
    ac.sendGoal(target)
    rospy.loginfo("You set new goal")

def cancel_goal_clicked(b): 
    ac.cancelGoal()
    rospy.loginfo("Goal cancelled")
    
set_goal.on_click(set_goal_clicked)
cancel_goal.on_click(cancel_goal_clicked)


Initialization of ROS communication

In [ ]:
ac = SimpleActionClient('/reaching_goal', PlanningAction)
ac.waitForServer()
client = rospy.ServiceProxy('/goal_info', Service1)

Cancel goal service

In [ ]:
def cancel_goal_service():
    goal = Service1Request()
    goal.goal_count = 1
    client.call(goal)
    
cancel_goal.on_click(cancel_goal_service())

Odometry

In [ ]:
def odom_callback(data):
    global robot_x, robot_y, robot_vel_x, robot_vel_y
    robot_x = data.pose.pose.position.x
    robot_y = data.pose.pose.position.y
    robot_vel_x = data.twist.twist.linear.x
    robot_vel_y = data.twist.twist.linear.y
    robot_position.value = f'x: {robot_x}, y: {robot_y}'

Subscribers

In [ ]:
rospy.Subscriber("/odom", Odometry, odom_callback)

Publishers

In [ ]:
def publish_robot_data():
    pub = rospy.Publisher('robot_data', Message1, queue_size=10)
    rate = rospy.Rate(10) # 10 Hz
    while not rospy.is_shutdown():
        msg = Message1()
        msg.x = robot_x
        msg.y = robot_y
        msg.vel_x = robot_vel_x
        msg.vel_y = robot_vel_y
        pub.publish(msg)
        rate.sleep()

Display widgets

In [ ]:
display(HBox([x_new, y_new, set_goal, cancel_goal]))
display(HBox([robot_position, target_position, distance_obstacle]))

Thread to publish robot data

In [ ]:
threading.Thread(target=publish_robot_data).start()

Plot position

In [ ]:
def plot_positions():
    fig, ax = plt.subplots()
    ax.plot([robot_x, x_new.value], [robot_y, y_new.value], 'ro-')
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_title('Positions of Robot and Target')
    plt.show()

plot_positions_button = widgets.Button(description='Plot positions')
plot_positions_button.on_click(plot_positions())
display(plot_positions_button)
